In [52]:
import numpy as np
import warnings
from google.colab import drive
drive.mount('/content/drive')
warnings.filterwarnings("ignore")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#daily_sales_numbers

In [53]:
import tensorflow as tf
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

# convert to tensorflow dataset
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [54]:
# iterate throw data

for sales in tf_dataset:
  print(sales.numpy())

print('*'*50)

for sales in tf_dataset.as_numpy_iterator():
  print(sales)

21
22
-108
31
-1
32
34
31
**************************************************
21
22
-108
31
-1
32
34
31


In [55]:
# first n element
for sales in tf_dataset.take(3):
  print(sales.numpy())

21
22
-108


In [56]:
# filter sales
tf_dataset = tf_dataset.filter(lambda x: x>0)

for sales in tf_dataset.as_numpy_iterator():
  print(sales)

21
22
31
32
34
31


In [57]:
# shuffle
tf_dataset = tf_dataset.shuffle(3)

for sales in tf_dataset.as_numpy_iterator():
  print(sales)

21
31
34
32
31
22


In [58]:
# Batch
tf_dataset = tf_dataset.batch(2)

for sales in tf_dataset.as_numpy_iterator():
  print(sales)

[21 22]
[34 31]
[32 31]


In [59]:
# in one line
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset = tf_dataset.filter(lambda x: x>0).shuffle(2).batch(2)

for sales in tf_dataset.as_numpy_iterator():
  print(sales)

[21 31]
[22 34]
[31 32]


#images

In [60]:
imgs_dataset = tf.data.Dataset.list_files('/content/drive/MyDrive/image_classification/test/*/*', shuffle=False)

len_imgs = len(imgs_dataset)
len_imgs

140

In [61]:
print(type(imgs_dataset))

<class 'tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset'>


In [62]:
for img in imgs_dataset.take(5):
  print(img)

tf.Tensor(b'/content/drive/MyDrive/image_classification/test/cats/cat_1.jpg', shape=(), dtype=string)
tf.Tensor(b'/content/drive/MyDrive/image_classification/test/cats/cat_106.jpg', shape=(), dtype=string)
tf.Tensor(b'/content/drive/MyDrive/image_classification/test/cats/cat_109.jpg', shape=(), dtype=string)
tf.Tensor(b'/content/drive/MyDrive/image_classification/test/cats/cat_113.jpg', shape=(), dtype=string)
tf.Tensor(b'/content/drive/MyDrive/image_classification/test/cats/cat_114.jpg', shape=(), dtype=string)


In [63]:
imgs_dataset = imgs_dataset.shuffle(200)

for img in imgs_dataset.take(5):
  print(img)

tf.Tensor(b'/content/drive/MyDrive/image_classification/test/dogs/dog_519.jpg', shape=(), dtype=string)
tf.Tensor(b'/content/drive/MyDrive/image_classification/test/dogs/dog_75.jpg', shape=(), dtype=string)
tf.Tensor(b'/content/drive/MyDrive/image_classification/test/dogs/dog_168.jpg', shape=(), dtype=string)
tf.Tensor(b'/content/drive/MyDrive/image_classification/test/cats/cat_585.jpg', shape=(), dtype=string)
tf.Tensor(b'/content/drive/MyDrive/image_classification/test/dogs/dog_196.jpg', shape=(), dtype=string)


In [64]:
class_name = ['cat', 'dog']

In [65]:
train_size = int(len_imgs*0.8)
train_dataset = imgs_dataset.take(train_size)
test_dataset = imgs_dataset.skip(train_size)

In [66]:
len(train_dataset), len(test_dataset)

(112, 28)

In [67]:
def get_lable(img_path):
  return "cat" if tf.strings.split(img_path,'/')[-2] == "cats" else "dog"

get_lable('/content/drive/MyDrive/image_classification/test/cats/cat_528.jpg')

'cat'

In [68]:
def process_img(img_path):
  lable = get_lable(img_path)
  img = tf.io.read_file(img_path)
  img = tf.image.decode_jpeg(img)
  img = tf.image.resize(img, [128, 128])
  return img, lable

In [69]:
img, lable = process_img('/content/drive/MyDrive/image_classification/test/cats/cat_528.jpg')
img.numpy()

array([[[ 50.34375 ,  34.34375 ,  18.34375 ],
        [ 51.971558,  36.065308,  20.034058],
        [ 60.59375 ,  45.4375  ,  29.15625 ],
        ...,
        [ 80.      ,  74.      ,  48.5625  ],
        [ 79.      ,  73.      ,  49.      ],
        [ 78.      ,  74.      ,  49.      ]],

       [[ 50.34375 ,  34.34375 ,  18.34375 ],
        [ 54.04297 ,  38.13672 ,  22.105469],
        [ 60.904663,  45.748413,  29.467163],
        ...,
        [ 80.02966 ,  74.02966 ,  48.592163],
        [ 79.003296,  73.003296,  49.003296],
        [ 77.89453 ,  73.89453 ,  48.89453 ]],

       [[ 50.34375 ,  34.34375 ,  18.34375 ],
        [ 54.9375  ,  39.03125 ,  23.      ],
        [ 61.15625 ,  46.      ,  29.71875 ],
        ...,
        [ 79.48694 ,  73.48694 ,  48.04944 ],
        [ 79.03125 ,  73.03125 ,  49.03125 ],
        [ 77.      ,  73.      ,  48.      ]],

       ...,

       [[178.6875  , 178.6875  , 170.6875  ],
        [185.53284 , 185.53284 , 177.53284 ],
        [180.89453 , 1

In [70]:
train_dataset = train_dataset.map(process_img)
test_dataset = test_dataset.map(process_img)

In [71]:
for img, lable in train_dataset.take(1):
  print(img)
  print(lable)

tf.Tensor(
[[[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 ...

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]], shape=(128, 128, 3), dtype=float32)
tf.Tensor(b'cat', shape=(), dtype=string)


In [72]:
# scale
def scale(image, label):
    return image/255, label

In [74]:
train_dataset = train_dataset.map(scale)

In [76]:
for image, label in train_dataset.take(1):
    print("****Image: ",image.numpy())
    print("****Label: ",label.numpy())

****Image:  [[[3.49203438e-01 3.25674027e-01 2.75428921e-01]
  [4.04532403e-01 3.73159856e-01 2.90557981e-01]
  [4.40073520e-01 4.16544110e-01 2.84156710e-01]
  ...
  [7.22303927e-01 4.68995094e-01 1.46953508e-01]
  [7.05390215e-01 4.78674561e-01 1.63233384e-01]
  [6.86090708e-01 4.23345596e-01 1.17463239e-01]]

 [[3.90808821e-01 3.71200979e-01 2.91053921e-01]
  [4.62745100e-01 4.36391324e-01 3.19374233e-01]
  [4.93321091e-01 4.74396825e-01 3.07640165e-01]
  ...
  [7.26348042e-01 4.79289204e-01 1.14583336e-01]
  [6.98039234e-01 4.35294122e-01 1.29105389e-01]
  [6.86274529e-01 4.23529416e-01 1.17647059e-01]]

 [[4.08026963e-01 3.92340690e-01 2.94301480e-01]
  [4.54779416e-01 4.31137025e-01 3.05985749e-01]
  [5.00367641e-01 4.83787149e-01 3.09999228e-01]
  ...
  [7.55392134e-01 4.86397058e-01 1.29534319e-01]
  [7.09926486e-01 4.55024511e-01 1.02083333e-01]
  [6.91806436e-01 4.29061353e-01 1.23178996e-01]]

 ...

 [[4.07843143e-01 4.82352942e-01 1.72303915e-01]
  [4.11887258e-01 4.8639705